In [9]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

#### Definições importantes que vão pro relatório
1. **Beta:** By definition, the market has a beta of 1.0, and individual stocks are ranked according to how much they deviate from the market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock's beta is less than 1.0. High-beta stocks are supposed to be riskier but provide a potential for higher returns; low-beta stocks pose less risk but also lower returns.

### Fuzzy 2 - Classificação do Grau de Risco de uma *stock*
1. **INPUTS**
    1. Média de Variação
        * Universo: Qual foi a média de variação no valor das *stocks* (em módulo)?
        * Conjunto Fuzzy: baixo, médio, alto
    2. Beta
        * Universo: Qual o valor da volatilidade da *stock* com relação a volatilidade do mercado?
        * Conjunto Fuzzy: baixo, alto
2. **OUTPUTS**
    1. Risco
        * Universo: Qual o grau de risco ao se investir nessa *stock* ?
        * Conjunto Fuzzy: baixo, médio, alto
3. **RULES**
    * Se a *stock* apresenta *baixa variação* e um *baixo valor beta* **ENTAO** o risco é baixo
    * Se a *stock* apresenta *baixa variação* e um *alto valor beta* **ENTAO** o risco é baixo (ou médio)
    * Se a *stock* apresenta *alta variação* e um *alto valor *beta* **ENTAO** o risco é alto
    * Se a *stock* apresenta *alta variação* e um *baixo valor *beta* **ENTAO** o risco é alto (ou médio)
    * **OBS: DA PRA PENSAR EM MAIS REGRA**